In [51]:
def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            # Special processing for the first line after "USD USD"
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])  # Preserve spaces between words
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])  # Preserve spaces between words
                
                # Further clean-up for parenthesis within the first sentence
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                # Strip double quotes if present
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            # Process subsequent lines
            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1
            
            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    return goods_descriptions


import os
import re
import pdfplumber
import pandas as pd
import fitz

pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"
    
            goods_descriptions = extract_goods_description(all_text)
            print(goods_descriptions)
        
        with fitz.open(pdf_path) as doc:
            PyMuPDF_text = ""
            for page in doc:
                PyMuPDF_text += page.get_text() + "\n"
            print(PyMuPDF_text)



['60% BCI COTTON 40%\nRECYCLE POST', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER']
INVOICE
Supplier:
IMPRESS-NEWTEX COMPOSITE TEXTILES LTD.
 Gorai Industrial Area, Mirzapur
1942 Tangail
Bangladesh
Invoice No and Date:
Number:2505657
Date: 13-05-2025
Exporters Ref:
S/C:INCTL/H&M/001/2025
DT:08-01-2025
EXP:1471-019366-2025
DT:13-05-2025
H&M Order No: 294225-5832
Product Id: 0967153
Country of Origin of Goods:
Bangladesh
Buyer:
H & M Hennes & Mauritz GBC AB
Mäster Samuelsgatan 46A
106 38 Stockholm
Sweden
Consignee:
H&M Hennes & Mauritz Uruguay S.A Guipuzcoa 331
Piso 3 Zip Code: 11300 Montevideo, Uruguay
Remarks:
Mode of Transport:
Sea
Final Destination:
Uruguay
UY
Terms of Delivery:FCA Chittagong
Port of Loading:
Chittagong
Port of Discharge:
Terms of Payment:
EOM+63 days
Warehouse ID
No of Pkgs
Description 

In [52]:
import os
import pdfplumber
import fitz  # PyMuPDF

def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1

            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    return goods_descriptions


pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        
        # Extract text dynamically
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"
        goods_descriptions = extract_goods_description(all_text)
        print("Extracted descriptions:", goods_descriptions)
        
        # Open PDF with PyMuPDF
        doc = fitz.open(pdf_path)
        for page in doc:
            for desc in goods_descriptions:
                # Some descriptions may span multiple lines; search line by line
                for line in desc.split('\n'):
                    areas = page.search_for(line.strip())
                    for area in areas:
                        page.add_redact_annot(area, fill=(1, 1, 1))  # white fill
            page.apply_redactions()
        
        # Save modified PDF
        new_pdf_path = os.path.join(pdf_dir, f"modified_{filename}")
        doc.save(new_pdf_path)
        doc.close()
        print(f"Processed and saved: {new_pdf_path}")


Extracted descriptions: ['60% BCI COTTON 40%\nRECYCLE POST', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER']
Processed and saved: C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test\modified_Invoice Edit Files (4).pdf


HardCoded Edit

In [65]:
import os
import fitz  # PyMuPDF

def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1

            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    return goods_descriptions

pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"
output_dir = os.path.join(pdf_dir, "Cleaned_PDFs")
os.makedirs(output_dir, exist_ok=True)  # create output folder if it doesn't exist

text_to_remove = """60% BCI COTTON 40%
RECYCLE POST"""

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        # Extract text dynamically
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"
        goods_descriptions = extract_goods_description(all_text)
        print("Extracted descriptions:", goods_descriptions)

        doc = fitz.open(pdf_path)
        
        for page in doc:
            # Find all instances of the text
            text_instances = page.search_for(text_to_remove)
            for inst in text_instances:
                # Draw white rectangle over the text
                page.draw_rect(inst, color=(1, 1, 1), fill=(1, 1, 1))
        
        # Save cleaned PDF to new file
        new_pdf_path = os.path.join(output_dir, f"cleaned_{filename}")
        doc.save(new_pdf_path)
        doc.close()
        
        print(f"Saved cleaned PDF: {new_pdf_path}")


Extracted descriptions: ['60% BCI COTTON 40%\nRECYCLE POST', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER']
Saved cleaned PDF: C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test\Cleaned_PDFs\cleaned_Invoice Edit Files (4).pdf


Dynamic Edit

In [64]:
import os
import fitz  # PyMuPDF
import pdfplumber

def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1

            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    print(f"Extracted descriptions: {goods_descriptions}")
    return goods_descriptions

pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"
output_dir = os.path.join(pdf_dir, "Cleaned_PDFs")
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        
        # Extract all text using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    all_text += page_text + "\n"
        
        # Get all goods descriptions dynamically
        goods_descriptions = extract_goods_description(all_text)
        print(f"Extracted {len(goods_descriptions)} descriptions from {filename}")
        
        # Open PDF with PyMuPDF
        doc = fitz.open(pdf_path)
        for page in doc:
            for desc in goods_descriptions:
                # Search and remove each description
                text_instances = page.search_for(desc)
                for inst in text_instances:
                    page.draw_rect(inst, color=(1, 0, 0), width=1)  
                    page.draw_rect(inst, color=(1,1,1), fill=(1,1,1))
        
        # Save cleaned PDF to new file
        new_pdf_path = os.path.join(output_dir, f"cleaned_{filename}")
        doc.save(new_pdf_path)
        doc.close()
        print(f"Saved cleaned PDF: {new_pdf_path}")


Extracted descriptions: ['60% BCI COTTON 40%\nRECYCLE POST', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER', '60% BCI COTTON 40%\nRECYCLE POST\nCONSUMER']
Extracted 6 descriptions from Invoice Edit Files (4).pdf
Saved cleaned PDF: C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test\Cleaned_PDFs\cleaned_Invoice Edit Files (4).pdf


Correctly cleaned

In [66]:
import os
import fitz  # PyMuPDF
import pdfplumber

def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1

            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    return goods_descriptions


pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"
output_dir = os.path.join(pdf_dir, "Cleaned_PDFs")
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)

        # Extract all text using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    all_text += page_text + "\n"

        # Get all goods descriptions dynamically
        goods_descriptions = extract_goods_description(all_text)
        print(f"Extracted {len(goods_descriptions)} descriptions from {filename}")

        # Open PDF with PyMuPDF
        doc = fitz.open(pdf_path)

        for page_index, page in enumerate(doc):
            if page_index < len(goods_descriptions):
                desc = goods_descriptions[page_index]
                text_instances = page.search_for(desc)
                for inst in text_instances:
                    page.draw_rect(inst, color=(1, 0, 0), width=1)
                    page.draw_rect(inst, color=(1, 1, 1), fill=(1, 1, 1))
                print(f"Cleared description from page {page_index+1}: {desc}")

        # Save cleaned PDF to new file
        new_pdf_path = os.path.join(output_dir, f"cleaned_{filename}")
        doc.save(new_pdf_path)
        doc.close()
        print(f"Saved cleaned PDF: {new_pdf_path}")


Extracted 6 descriptions from Invoice Edit Files (4).pdf
Cleared description from page 1: 60% BCI COTTON 40%
RECYCLE POST
Cleared description from page 2: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
Cleared description from page 3: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
Cleared description from page 4: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
Cleared description from page 5: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
Cleared description from page 6: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
Saved cleaned PDF: C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test\Cleaned_PDFs\cleaned_Invoice Edit Files (4).pdf


Added Validations 

In [67]:
import os
import fitz  # PyMuPDF
import pdfplumber

def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1

            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    return goods_descriptions


pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"
output_dir = os.path.join(pdf_dir, "Cleaned_PDFs")
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)

        # Extract all text using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    all_text += page_text + "\n"

        # Get all goods descriptions dynamically
        goods_descriptions = extract_goods_description(all_text)
        print(f"Extracted {len(goods_descriptions)} descriptions from {filename}")

        # Open PDF with PyMuPDF
        doc = fitz.open(pdf_path)

        # Validation: descriptions count must match page count
        if len(goods_descriptions) != len(doc):
            print(f"⚠️ Skipping {filename}: pages={len(doc)} but descriptions={len(goods_descriptions)}")
            doc.close()
            continue

        for page_index, page in enumerate(doc):
            desc = goods_descriptions[page_index]
            text_instances = page.search_for(desc)
            if not text_instances:
                print(f"⚠️ Description not found on page {page_index+1}, skipping...")
                continue
            for inst in text_instances:
                page.draw_rect(inst, color=(1, 0, 0), width=1)
                page.draw_rect(inst, color=(1, 1, 1), fill=(1, 1, 1))
            print(f"✅ Cleared description from page {page_index+1}: {desc}")

        # Save cleaned PDF to new file
        new_pdf_path = os.path.join(output_dir, f"cleaned_{filename}")
        doc.save(new_pdf_path)
        doc.close()
        print(f"💾 Saved cleaned PDF: {new_pdf_path}")


Extracted 6 descriptions from Invoice Edit Files (4).pdf
✅ Cleared description from page 1: 60% BCI COTTON 40%
RECYCLE POST
✅ Cleared description from page 2: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
✅ Cleared description from page 3: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
✅ Cleared description from page 4: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
✅ Cleared description from page 5: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
✅ Cleared description from page 6: 60% BCI COTTON 40%
RECYCLE POST
CONSUMER
💾 Saved cleaned PDF: C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test\Cleaned_PDFs\cleaned_Invoice Edit Files (4).pdf


Replace Empty Places with Texts

In [78]:
import os
import fitz  # PyMuPDF
import pdfplumber

def extract_goods_description(text):
    goods_descriptions = []
    lines = text.split('\n')
    found_usd_usd = False
    i = 0
    while i < len(lines):
        line = lines[i]
        if 'USD USD' in line:
            found_usd_usd = True
            i += 1
            continue
        if found_usd_usd:
            description_parts = []
            if i < len(lines):
                words = lines[i].split()
                if len(words) > 3:
                    first_sentence = ' '.join(words[:-3])
                else:
                    i += 1
                    if i < len(lines):
                        words = lines[i].split()
                        first_sentence = ' '.join(words[:-3])
                first_sentence = first_sentence.split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                if first_sentence.startswith('"'):
                    first_sentence = first_sentence[1:].strip()
                description_parts.append(first_sentence)
                i += 1

            while i < len(lines) and not lines[i].startswith("Container"):
                current_sentence = lines[i].split('PCS/PACK')[0].split('(')[0].split(')')[0].strip()
                description_parts.append(current_sentence)
                i += 1

            description = "\n".join(description_parts).strip()
            if "Container" in description:
                description = description.split("Container")[0].strip()
            if "HS Code:" in description:
                description = description.split("HS Code:")[0].strip()
            goods_descriptions.append(description)
            found_usd_usd = False
        else:
            i += 1
    return goods_descriptions


pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test"
output_dir = os.path.join(pdf_dir, "Cleaned_PDFs")
os.makedirs(output_dir, exist_ok=True)

# New description to insert
new_description = "20% Recycled pre consumer\nCotton+ 80% cotton"

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)

        # Extract all text using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    all_text += page_text + "\n"

        # Get all goods descriptions dynamically
        goods_descriptions = extract_goods_description(all_text)
        print(f"Extracted {len(goods_descriptions)} descriptions from {filename}")

        # Open PDF with PyMuPDF
        doc = fitz.open(pdf_path)

        # Validation: descriptions count must match page count
        if len(goods_descriptions) != len(doc):
            print(f"⚠️ Skipping {filename}: pages={len(doc)} but descriptions={len(goods_descriptions)}")
            doc.close()
            continue

        for page_index, page in enumerate(doc):
            desc = goods_descriptions[page_index]
            text_instances = page.search_for(desc)
            if not text_instances:
                print(f"⚠️ Description not found on page {page_index+1}, skipping...")
                continue

            # Clear all rectangles first
            for inst in text_instances:
                page.draw_rect(inst, color=(1, 0, 0), width=1)
                page.draw_rect(inst, color=(1, 1, 1), fill=(1, 1, 1))

            # Insert new description only once at the first rectangle
            first_inst = text_instances[0]
            lines = new_description.split('\n')
            y_offset = 0
            for line in lines:
                page.insert_text(
                    (first_inst.x0, first_inst.y0 + y_offset + 8),
                    line,
                    fontsize=7.5,
                    color=(0, 0, 0)
                )
                y_offset += 9  # adjust line spacing

            print(f"✅ Cleared and inserted new description on page {page_index+1}")


        # Save cleaned PDF to new file
        new_pdf_path = os.path.join(output_dir, f"cleaned_{filename}")
        doc.save(new_pdf_path)
        doc.close()
        print(f"💾 Saved cleaned PDF: {new_pdf_path}")


Extracted 6 descriptions from Invoice Edit Files (4).pdf
✅ Cleared and inserted new description on page 1
✅ Cleared and inserted new description on page 2
✅ Cleared and inserted new description on page 3
✅ Cleared and inserted new description on page 4
✅ Cleared and inserted new description on page 5
✅ Cleared and inserted new description on page 6
💾 Saved cleaned PDF: C:\Users\Altersense\Desktop\ERP-RPA\inv_Edit_R&D\Test\Cleaned_PDFs\cleaned_Invoice Edit Files (4).pdf
